In [1]:
from PyNetwork import utils
import pyopencl as cl
import pyopencl.array as cl_array
import numpy as np

n = 10
m = 100
d = 20

platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

In [2]:
c_code = '''
__kernel void cut(__global float *a, __global float *out, int original_width, int output_width){
    int global_col = get_global_id(1);
    int global_row = get_global_id(0);

    out[global_row * output_width + global_col] = a[global_row * original_width + global_col];
}
'''

In [3]:
program = cl.Program(context, c_code).build()

In [9]:
def cut(X, n, m):
    out = cl_array.zeros(queue, (n, m), dtype=np.float32)
    program.cut(queue, (n, m), None,
                X.data, out.data, np.int32(X.shape[1]), np.int32(m)).wait()
    return out

In [14]:
X = np.random.rand(200, 100).astype(np.float32)
X_gpu = cl_array.to_device(queue, X)

In [18]:
n, m = np.random.randint(200), np.random.randint(100)
out = cut(X_gpu, n, m)

np.testing.assert_almost_equal(X[:n, :m], out.get())

In [12]:
np.testing.assert_almost_equal(X[:n, :m], out.get())